# Finding Parler usernames on Twitter

Part of the goal of this project is to analyze social media behavior between the two platforms: Parler and Twitter. To do this, in a previous notebook, I filtered the original Parler user dataset to create two subsets of data. The first includes users that mention support for or an affiliation with a far-right militia group in their Parler bios, and the second focuses on users who mentioned being banned from another social media platform in their Parler bios. I also included users verified as public figures on Parler.

In this notebook, I am running the usernames from each subset to generate a list of accounts with the same username on Twitter. Previous academic studies have noted that people often use the same usernames or handles across platforms: https://dl.acm.org/doi/pdf/10.1145/2487575.2487648, https://arxiv.org/pdf/1902.08737.pdf. My primary goal is to determine which users with the same Twitter and Parler handles have active or suspended accounts.

I am also pulling additional information for those with active Twitter accounts, including follower counts and Twitter bios.

Before running the dataset through the API, I manually verified that each user's bio indicated one of the groups listed above.

# Importing tools

In [1]:
import requests
import os
import json
import pandas as pd
pd.set_option('display.max_columns', 500)
import time
import datetime

/Users/sheridanwall/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Importing the data

Import either the right_short_users.csv or the suspend_short_users.csv here.

In [25]:
banned_short_users = pd.read_csv('banned_short_users.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'banned_short_users.csv'

# Setting up Twitter API

Code to interact with the API taken directly from here: https://github.com/twitterdev/Twitter-API-v2-sample-code

In [ ]:
def auth():
     return os.environ.get("BEARER TOKEN")

In [13]:
def create_url(USERNAME):
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
    usernames = "usernames="+USERNAME
    user_fields = "user.fields=description,created_at,entities,public_metrics"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url

In [14]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [15]:
def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
#     if response.status_code != 200:
#         #raise Exception(
#             "Request returned an error: {} {}".format(
#                 response.status_code, response.text
#             )
#         )
    return response

# Testing with single username

In [16]:
bearer_token = ""
url = create_url("")
headers = create_headers(bearer_token)
response = connect_to_endpoint(url, headers)
print(json.dumps(response.json(), indent=4, sort_keys=True))

200
{
    "data": [
        {
            "created_at": "2009-03-11T00:11:59.000Z",
            "description": "",
            "id": "23687920",
            "name": "Joe Perez",
            "public_metrics": {
                "followers_count": 9,
                "following_count": 14,
                "listed_count": 0,
                "tweet_count": 1139
            },
            "username": "jperez3"
        }
    ]
}


# Retrieving account status & saving to text file

In [17]:
def get_title_and_detail(response):
    if response.status_code==200:
        if 'errors' in response.json():
            title = response.json()['errors'][0]['title']
            detail = response.json()['errors'][0]['detail']
        else:
            title = 'account exists'
            detail = ''
    else:
        title = response.json()["title"]
        detail = response.json()["detail"]
    return title, detail

In [11]:
with open("banned_api_results.txt", "w+") as f:
    for user in banned_short_users['username'].values:
        print(user)
        url = create_url(user)
        response = connect_to_endpoint(url, headers)
        if response.status_code == 429:
            wait_until = datetime.datetime.utcfromtimestamp(int(response.headers["x-rate-limit-reset"]))
            print("Got 429 code, waiting until {}".format(wait_until))
            time_to_wait = wait_until - datetime.datetime.utcnow()
            # print(time_to_wait.seconds)
            time.sleep(time_to_wait.seconds * 1.5)
            response = connect_to_endpoint(url, headers)
            title, detail = get_title_and_detail(response)
        else:
            title, detail = get_title_and_detail(response)
        
        f.write("{}|{}|{}|{}\n".format(user, response.status_code, title, detail))

ronnieOD
200
poetWOAgun
200
Somaspecialist
200
DapiceDom
200
Rossicheryl
200
stefierussell
200
Sammyfalcon69
200
Political1
200
DesertStormGirl
200
Paulmac708
200
Jackson5868
200
Geocorb
200
nuzombie4
200
Singlehappylady
200
Lyndzarelli
200
Dbf752
200
KerryFrench
200
BradAllen2020
200
Rmbforkmg20
200
GB1976
200
Ransan89
200
honoluluchris
200
BruceKent
200
Racheljean29
200
Wakewakingwoke
200
Beast50
200
Samcon42
200
Drphyto
200
ARIZQNA
200
NinjaMouse
200
AKrupka
200
Louisdi0308
200
ToeRogan
200
Spoiler24
200
Qtpie4truth
200
DonWilson69
200
Jschvat
200
GuaranteedEtern
200
LadyLove
200
DanthemanMAGA
200
RosaryCrusader
200
Samarasusan9
200
bbdoll11
200
Kupchurch45
200
TripSlater
200
kirbizzle
200
Foolme1ce
200
Paulfortuin
200
Poodleface
200
UltraLord
200
Txbuyer
200
HenryHughes
200
dman767
200
emo50yearold
200
Decentralized20
200
PFD1961
200
CoreyaDevers
200
TheLife
200
W1LL1E
200
Flyfishinmary
200
BalzarFrozenov
200
Budhenry79
200
Trump4everrr
200
Sherrihamaker
200
Johnsdroid46
200
Shonna

200
Anthonywk419
200
PauliGMac
200
DarlingDeb
200
Maschmelzer65
200
0live0il
200
Dontgiveup1
200
JonJones5584369
200
Lovey
200
deebee2017
200
CBDPete
200
Randog76
200
JDMUnderground
200
JasonJohnzzz
200
Codezilla11
200
Shamusod18
200
SaltyLime
200
Lzmallin
200
BackUpDonTrump
200
NiamhKousaka
200
fatkid7813
200
ModerateOkie
200
Bigbopper9099
200
TheAaronWilhelm
200
Hopefulforgood
200
Hotoiler2009
200
cyclemadster
200
Bakunin420
200
JTMsish
200
BassAndDrumz
200
Planohog
200
prakashkhunti
200
seanmiller555
200
Irvin4rose
200
Nhhsgal
200
BradHutchings
200
PatriotPopeye
200
RickWimmer0919
200
SandeepNair
200
Freddebergkvist
200
Woofingson
200
Gershk
200
skvmfvck
200
FocusST170SVT
200
Timdelauter
200
Phil4Seattle
200
UranusHasRings
200
Walkerstandup
200
ElectedMob
200
Sjcrowther
200
Notdecided
200
zzzfa20dit
200
JiuJitsuDweeb
200
NiteSights
200
Icanc4milz
200
Smiffee
200
apseheath
200
Csablades
200
LogosWalker
200
lowsey51
200
Djshadowsix
200
MrMeat
200
Pumpkinhaid
200
AllieSF
200
SmokeMSJ
2

200
DewieLovesSugar
200
QuintanaDaTruth
200
THEInfamousT
200
DebbieGarnet
200
ConservFem
200
AxeTruth
200
JoshPaul
200
MyPresIsDJT
200
icufos
200
AM4Trump2020
200
GregmhKs
200
KBish1166
200
Pam3030
200
AllPeopleUSA
200
Mrpizzle77
200
MsDubson
200
user3743
200
RyDogs845
200
CCRyder1776
200
Robinholmes
200
iMeltSn0wflakes
200
Birks31
200
Winhgj
200
PseudoEpic
200
Melancholalia
200
jasonclarke
200
Ohour1
200
GlennOstrosky
200
Aaronwalsh1979
200
JB1992
200
Molsen10mm
200
FloridaThreeper
200
TMJW
200
Terrazalea
200
Ktc8881
200
Danspomer
200
RobRhone9
200
Valuator44
200
DylanWheeler511
200
PSYCOHOLIC
200
SarahsCunnin
200
Vapor069
200
HelianthusDaze
200
Knuckles
200
BigBaldSexy
200
Alex305
200
Crila1612
200
YumaJoy
200
RealRedRepublic
200
PluckyBrit
200
Angel_johnson27
200
sarbearmsp
200
BruceL6901
200
Nathalie
200
Dod4734
200
0xeb0de
200
Dshep678
200
kelseywangshol
200
roseMAGA
200
PirateMike
200
Hillbilltaco
200
ProactiveBroker
200
Kowalski38
200
Bres92
200
LindyWise
200
ConTrumptible
200
z

200
RogersJoe97
200
dontdoxmebro
200
McGunner
200
Slugglife
200
MatthewMagnee
200
Desertfoxnick
200
Msabo1967
200
MrRichardSmoker
200
willibebanned
200
OlafDtherit
200
Mwakefield92
200
cheaptrixx
200
SnookDaddy
200
Onebawcity
200
Storywrnr50
200
Texiled
200
RobMack
200
glcmom
200
GinaGrin2016
200
Geekyblonde
200
Sjeetla
200
oneshot2
200
MAGAZombie
200
N9tas
200
StellaMaris
200
yelmandolini
200
roybaen1
200
pedaltheworld
200
Thundar
200
Dsknight3
200
reagansays
200
Daveross3476
200
PaulAloise
200
DynaLee
200
SAARevolver
200
Hollyhenderson1
200
Alangaylord
200
Jeromeyee
200
dditaranto
200
KimmyKesler
200
DumpCuomoQ2022Q
200
GOLENO
200
Steel1234
200
TheEyesofTexas
200
Wolf10
200
LeviGuy
200
BedroomJunkie
200
ctsa
200
ViktoriaColvin
200
TheDamnedYankee
200
Idahogreg
200
Racerxrp
200
DVGCPresident
200
Holyhills
200
Billpyne
200
WWG1WGALL
200
uptheriver
200
Neotribal
200
Searay2005
200
SupportOurLefty
200
courtscott
200
Richardsonb0013
200
RNDL
200
realkingrobbo
200
Pencey21
200
Descass
200


200
wokingteaguy
200
2020Qanon
200
FrankStein
200
Integritygamer0
200
Amymek
200
Cubansam32164
200
Kektyrant
200
RemyLeBeauPhD
200
aJerseyGirlYall
200
hendrikmaass
200
Lars22
200
JManke
200
dcosta77
200
Clubcaddy16
200
TheGreatDaneDad
200
Tanithcervin
200
Nickwerling
200
Voofie4711
200
adequatelylucid
200
WhiteJaws
200
TesticularFred
200
Cmaree
200
Garywerbisky
200
MoryV
200
Mattdaltonrules
200
Fergo
200
ForgetTheLies
200
Amiewielandt
200
Tdb920
200
DanteLoSono
200
JosephKlein21
200
DustyRae1776
200
TobyM
200
CrayonRoyal
200
Sajuuk
200
KyleHarty666
200
Ringapoochnut23
200
Wrbodden
200
AlLupo
200
Khoffman1996
200
orsricofoak1
200
Davidbooth401
200
Coolbert
200
ZackMeloon
200
FireLordofUdun
200
AlexWars
200
HazyMcVay
200
TheTedStryker
200
AugustusAetius
200
Helendow
200
caugustine360
200
Jlozinsk
200
CarlosKillface
200
Talcum69
200
Conshinz
200
economyvoter
200
Jiggy175
200
Arvinds
200
Grebnitty1980
200
Kaosbred
200
RealCourtneyM
200
Travlr009
200
LuiSugalo
200
HudsonRoss1776
200
Riverst

200
awill
200
Johnhalgren
200
Williwonka
200
Rbccspicer71
200
Wkennedy17
200
xSHARKYx416
200
Presscalifornia
200
Realtalkbtr
200
Cuetheangel
200
Vipyr
200
MichaelShtalman
200
spoons
200
TopShotDustin
200
dontaskwhy2day
200
Nine0001
200
TheOchsReport
200
Sheltonusa
200
CarSales
200
FredricBeckman
200
EricWood
200
1Shred
200
ckennison47
200
godaiko2016
200
Tnydiaz1
200
Troyeccles1
200
LunaVader
200
SaveTheKulaks
200
ElyM
200
DigDoug
200
neal
200
Jms5303
200
AKMessiah
200
Brokegunboi
200
McGibner
200
JewishOdysseus
200
ABeachGirl
200
DrSledge
200
Fourdoor
200
Seanroper1
200
Corygroshek
200
Blaind166
200
OllieTNR
200
franke299
200
Dontlookback
200
LBananaRama
200
Washnative777
200
cfulgenzi
200
Keef2
200
JimmyConway17
200
DimmsSuck
200
NORDBLOD
200
Siprti3
429
Got 429 code, waiting until 2021-08-10 14:32:48
200
QWORLDORDER17
200
JHP
200
Grumpyolman21
200
Boxer
200
USMCpatriot
200
DarenevansUs
200
Warpath72
200
IAm5D
200
Benjamin330
200
gloriahilley
200
Nalacamp
200
MichellAnn
200
BUCKoFIVE

200
Jdk4jesus
200
ChaseShepherd77
200
Johngarrett911
200
Endgunfreezonz
200
whitedove2
200
EVHarris22
200
Tomcochran00
200
michaeljeaton
200
greenbayfan21
200
traciloveswords
200
Jjstyxdrummer
200
Mauiguy
200
mzb1
200
TomQCat
200
RealJohnnyT
200
rptc4
200
Effy
200
marionschafter
200
GregoryOverholt
200
Careylou114
200
RoseAlberta1
200
RDCII
200
redhead5830
200
SupaStarDad
200
OhmanThatsBeefy
200
PR20906
200
AdamDeane420
200
NATOmonitor
200
WyzeWildfire
200
RoyRogue25
200
JohnnyLanson
200
StillthesameCat
200
RosenRitter
200
mommyQ
200
TeresaTitsworth
200
Buckeyehillfarm
200
OzTrump1111
200
BandOnTwitter
200
Devito1993
200
DownTheRange
200
BhagwaDhari
200
FarinaJohnny
200
PollyOH
200
DrSamDavis
200
Alwaysinabind
200
JuanK
200
janlange
200
teamlinwood
200
Taty815
200
Schimpffj
200
Sdmetsfan1
200
Civics
200
Scodogno
200
Badriffrat
200
MatiHari
200
Cepoorbaugh
200
RightWingerMike
200
HarleyriderHD
200
proudnyvet
200
TRISHAJoy
200
Seedofabe
200
MicMaven
200
GabeKotter
200
FreakWhisperer
200


200
WWG1WGA101
200
GBPtbo1970
200
Dswepston
200
mfstratton
200
HOWDAREYOUU
200
64Delta
200
TrumpetOfGod1
200
Bob8967
200
Gloriasmith963
200
AnitaMcEgg
200
0vangel0
200
Bocajlaer
200
Patriottttpatsy
200
rocket44
200
AwakeningAngell
200
quazo17
200
Mtbinary
200
linwoodgroup
200
Swampysghost
200
meinhtx
200
Davidsport30
200
vengeful1
200
Shark154
200
Robert55
200
Trumpatriot45
200
Vogie2013
200
CC370Z
200
Cat09tails
200
Shellybs
200
Dianechick
200
kaylarose
200
WhiteBoyRick
200
HazilEye
200
SuzannaQ7
200
ArtimusMaximus
200
Jerzygma
200
NotADemonrat
200
VoltaireWeeb
200
ItsYaBoiPhiill
200
MemphisGrits
200
DeplorableLisa
200
ApeBass
200
RagedCitizen
200
Ihatetcook
200
nannasaeta
200
CurlyGirlNY
200
Flynhog1961
200
JBDude
200
jtjeremybtc
200
Bimingxian
200
HollywoodRed6
200
MikeMartin1733
200
stanislavpigol
200
TwitterSucks1
200
Tac1946
200
spirituality
200
Snoopywood1
200
TexasTumblewee3
200
Torregloria22
200
Flynguy66
200
thisperfectday1
200
JustPlainAnon
200
Nyc2ncstate
200
Jameslovett69


# Merge datasets and eliminate duplicates

Take the data from the text files and create CSVs. Import the CSVs into their corresponding notebooks and merge with the final subset dataframes. Eliminate any duplicate users. Create a dataframe of users with active Twitter accounts and export it as a CSV.

# Additional Twitter account information for active users

For those usernames with corresponding active Twitter accounts, I used Twitter API to pull their Twitter bios, follower counts and tweet counts to compare their behavior on Parler versus Twitter and to verify that they are the same users.

In [21]:
banned_active_users = pd.read_csv('banned_active_users.csv')

In [22]:
def get_user_description(response):
    if 'data' in response.json().keys():
        for desc in response.json()['data']:
            bios = desc['description']
            usernames = desc['username']
    else:
        bios = None
        usernames = None
    return bios, usernames

In [23]:
def get_user_metrics(response):
    if 'data' in response.json().keys():
        for desc in response.json()['data']:
            metrics = desc['public_metrics']
            followers = metrics['followers_count']
            following = metrics['following_count']
            listed = metrics['listed_count']
            tweets = metrics['tweet_count']
    else:
        metrics=None
        followers=None
        following=None
        listed=None
        tweets=None
    return metrics, followers, following, listed, tweets

In [24]:
with open("banned_active_api_results.txt", "w+") as f:
    for user in banned_active_users['username'].values:
        print(user)
        url = create_url(user)
        response = connect_to_endpoint(url, headers)
        if response.status_code == 429:
            wait_until = datetime.datetime.utcfromtimestamp(int(response.headers["x-rate-limit-reset"]))
            print("Got 429 code, waiting until {}".format(wait_until))
            time_to_wait = wait_until - datetime.datetime.utcnow()
            time.sleep(time_to_wait.seconds)
            response = connect_to_endpoint(url, headers)
            bios, usernames = get_user_description(response)
            metrics, followers, following, listed, tweets = get_user_metrics(response)
        else:
            bios, usernames = get_user_description(response)
            metrics, followers, following, listed, tweets = get_user_metrics(response)
        f.write("{}|{}|{}|{}|{}|{}|{}|{}\n".format(user, response.status_code, bios, usernames, metrics, followers, following, listed, tweets))

ronnieOD
200
poetWOAgun
200
Somaspecialist
200
Sammyfalcon69
200
Political1
200
Paulmac708
200
Geocorb
200
Singlehappylady
200
Lyndzarelli
200
Dbf752
200
KerryFrench
200
GB1976
200
honoluluchris
200
BruceKent
200
Racheljean29
200
Beast50
200
NinjaMouse
200
AKrupka
200
DonWilson69
200
LadyLove
200
RosaryCrusader
200
kirbizzle
200
Poodleface
200
UltraLord
200
Txbuyer
200
HenryHughes
200
emo50yearold
200
Decentralized20
200
TheLife
200
W1LL1E
200
Sherrihamaker
200
ShonnaUpton
200
ScrumpTheTexan
200
NanaSparkle
200
DadeD
200
Shadow1120
200
healthcaretruth
200
Billyjayp
200
ckm
200
inthecooler
200
donnieofthedead
200
MAGAmom5
200
Lexsayshello
200
Optimusprimal
200
dwknox
200
DanielBotha1
200
LindaOC
200
Stay2talk
200
LauraSteele
200
Jefffuss
200
Carl1258
200
Elmosdad
200
Devildog771
200
Dogman07
200
Tambourineman
200
Ketnavo
200
HippieChick
200
Rockstar20
200
Munterrocko
200
GreenQueen
200
LiberalMonkee
200
HelenBack1
200
davesrebellion
200
Daveremick
200
Pennysplace
200
duffingitup
200
poi

200
Covert
200
Dmare007
200
IMDeplorable3
200
Jlgottfred
200
Dutchtesterik
200
Lilykate
200
readyplayer1
200
Ichris05
200
Ksuchecka
200
Cielle
200
Chupacabra50
200
Koshernews
200
BigSteve666
200
OctDave
200
versecannon
200
THELASTOFUZ
200
CageyOne
200
Gpulse
200
heatherscorner
200
KTrulz
200
Karilalm
200
Browneyegal
200
TomHagen51
200
Jojo1963
200
Sharmccut48
200
DavidMendoza
200
Agias
200
GeorgiaGirl2
200
Ralphyounes
200
RobinOfLoxley
200
USAInfidel
200
Endtheswamp1
200
mjinnocent
200
MikeAtchison
200
JTINC
200
brittanywalker
200
JustinS57300833
200
RebelRuss
200
Samanthajohnson
200
RangerMama
200
DanBushey
200
GeorgiaSweetPea
200
Aftermath0
200
Johnfsmithjr1
200
ThomasPaine1776
200
GaryBear86
200
WarriorPrincess
200
BlackDogTeam
200
SunnyArizona1
200
JoeCater
200
Mcolwell13
200
deploriusBjG
200
Publiusvalerius
200
GetSome17
200
Ziplok
200
Spenceuk
200
Jimmyvegas
200
Plissken
200
PinkSquid
200
BlackBirdRayven
200
DavidAlexanders
200
Jojijay
200
BixlerJames
200
Steven62
200
Jery77
200


200
Lzherka
200
cannapatriot
200
cybervision
200
Resistance85
200
PatriotHearts
200
CitizenX
200
Drewblair
200
PatriotPapa1
200
Douge08
200
RedStatesRule
200
Freejack
200
Flatspin
200
Wolfman111
200
CurrencyDesign
200
DJ1984
200
ComputerJanitor
200
lefgavex
200
LastGreyWolf3
200
JaimaJay
200
david101
200
JustinJames
200
MarkWhitfield
200
RAFletch
200
Beavis17
200
GAStopTheSteal
200
SamTrinity
200
Eye4bass
200
findthetruthXX
200
Gingerbread91
200
Leaspeed6
200
Tib0r
200
Nemotoons
200
WilliamPaxton
200
Ghostgurl
200
richardbspencer
200
Angrygay
200
Troopersgal
200
TINIEBLAS
200
EleonoraFall
200
DigitalTruth
200
Tglass3
200
fuckingdinosaur
200
TANGO9
200
MattLipton
200
1984equals1776
200
OgerTheOgre
200
BrianWhiddon
200
TheRealMW
200
LilDeadst4r
200
Jcreidler
200
20dennis
200
codydchambers
200
BallSackBill
200
SpencerParr
200
AssFace
200
RoBough
200
Littleyoshi86
200
spiral7
200
Romi92
200
IPFreelee
200
GeoWalters
200
RichardQSpencer
200
Tharrison004
200
Madmaxxx
200
Billpierson1
200
Sikr

200
NewsOnTheFringe
200
Gildor
200
MalibuJohn
200
Lattee01
200
Magnum44
200
DaveAllen
200
StacyG
200
CrystalDawn
200
Dswepston
200
mfstratton
200
HOWDAREYOUU
200
64Delta
200
Bob8967
200
0vangel0
200
Bocajlaer
200
linwoodgroup
200
vengeful1
200
Shark154
200
Robert55
200
Cat09tails
200
Shellybs
200
Dianechick
200
kaylarose
200
WhiteBoyRick
200
DeplorableLisa
200
ApeBass
200
RagedCitizen
200
Ihatetcook
200
JBDude
200
spirituality
200
Snoopywood1
200
thisperfectday1
200
Djpublius
200
72BarbaraTx
200
CowboyKen
200
EMSDave
200
Languser
200
Simpleparadox
200
mattdylan
200
Wizard009
200
Darkwing7
200
Praffie
200
emmanuelusa
200
pattyfreeman
200
Bricktop
200
searobin
200
Bjsgarda317
200
Dcbw
200
Phoenix77Rising
200
ScottQ
200
downhomewoman
200
Mcroff58
200
CrazyUncle
200
MissM
200
FreeWill
200
realGregSmith
200
rebeccabrett
200
Napper
200
Garetttheferret
200
WhimsyDays
200
Sfrisco08
200
RobertWood55
200
AZTECA
200
ocsisters
200
MattStephenson
200
Elchingarito
200
TraceyBenna
200
Harty
200
Ameri

# Repeat process for both subsets and create CSVs from the text files